In [1]:
from ABDB import database as db
import matplotlib.pyplot as plt
import numpy as np
from retrain_ablooper import *
import torch
import pandas as pd
from rich import print as pprint
from ABlooper import CDR_Predictor

/Users/fabian/miniconda3/envs/auto-db-pipeline/lib/python3.9/site-packages/Bio/SubsMat/__init__.py:126: BiopythonDeprecationWarning: Bio.SubsMat has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.substitution_matrices as a replacement, and contact the Biopython developers if you still need the Bio.SubsMat module.
  warnings.warn(


Cannot do refinement


Own code for relaxing sturcutres

In [2]:
# torch settings
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.set_default_dtype(torch.float)

model = MaskDecoyGen(decoys=5).to(device = device).float()
model.load_state_dict(torch.load('best_models/best_model-2804-Radam-5-2optim', map_location=torch.device(device)))

batch_size = 1

train = torch.load('train_data/train.pt')
validation = torch.load('train_data/val.pt')
test = torch.load('train_data/test.pt')

val_dataloader = torch.utils.data.DataLoader(validation[:2], 
                                             batch_size=batch_size,
                                             num_workers=1,
                                             shuffle=False,
                                             pin_memory=True,
                                             )

test_dataloader = torch.utils.data.DataLoader(test, 
                                              batch_size=batch_size,
                                              num_workers=1,
                                              shuffle=False,
                                              pin_memory=True,
                                              )

In [3]:
for i in range(5):
    tmp = MaskDecoyGen(decoys=1).to(device = device).float()
    dict=torch.load("best_models/best_model-0305-Radam-1-2optim-"+str(i+1), map_location=torch.device(device))
    tmp.load_state_dict(dict)
    weights = tmp.blocks[0].state_dict()

    model.blocks[i].load_state_dict(weights)

In [4]:
cdr_rmsds, CDR_rmsds_relaxed, decoy_diversities, pdb_ids = produce_full_structures_of_val_set(val_dataloader, model, outdir='test', relax=False)

predict val set ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   0% -:--:--

In [5]:
cdr_rmsds

[[0.6297390460968018,
  0.5209707617759705,
  1.3106627464294434,
  0.3243178725242615,
  0.2549811899662018,
  0.37337222695350647],
 [0.2596547603607178,
  0.7193145751953125,
  2.2057271003723145,
  0.8642493486404419,
  0.32617682218551636,
  0.9799609780311584]]

In [6]:
CDR_rmsds_relaxed

[]

Calculate rmsds of relaxed structures

In [1]:
import os
from ABDB import database as db
import matplotlib.pyplot as plt
import numpy as np
from retrain_ablooper import *
import torch
import pandas as pd
from rich import print as pprint
from ABlooper import CDR_Predictor
from einops import rearrange

/Users/fabian/miniconda3/envs/auto-db-pipeline/lib/python3.9/site-packages/Bio/SubsMat/__init__.py:126: BiopythonDeprecationWarning: Bio.SubsMat has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.substitution_matrices as a replacement, and contact the Biopython developers if you still need the Bio.SubsMat module.
  warnings.warn(
Database path /Volumes/LaCie/sabdab-sabpred/data/ABDB was not found.


Cannot do refinement


In [2]:
files = os.listdir('/Users/fabian/Desktop/2804-RAdam-5-2optim-test')
files

['2ypv-HL-true.pdb',
 '2v17-HL-relaxed.pdb',
 '2vxv-HL-true.pdb',
 '1fns-HL.pdb',
 '3oz9-HL.pdb',
 '2v17-HL-true.pdb',
 '2e27-HL-true.pdb',
 '3gnm-HL-true.pdb',
 '4hpy-HL.pdb',
 '3oz9-HL-relaxed.pdb',
 '3g5y-BA-relaxed.pdb',
 '1gig-HL-true.pdb',
 '4f57-HL-true.pdb',
 '3g5y-BA.pdb',
 '2fb4-HL-relaxed.pdb',
 '3t65-BA-relaxed.pdb',
 '3eo9-HL.pdb',
 '2xwt-AB-relaxed.pdb',
 '3hc4-HL.pdb',
 '4h20-HL.pdb',
 '1fns-HL-true.pdb',
 '3t65-BA-true.pdb',
 '3ifl-HL-relaxed.pdb',
 '4nzu-HL.pdb',
 '2r8s-HL-relaxed.pdb',
 '1oaq-HL.pdb',
 '1oaq-HL-true.pdb',
 '2w60-AB-relaxed.pdb',
 '1mfa-HL-relaxed.pdb',
 '3hc4-HL-relaxed.pdb',
 '1mlb-BA.pdb',
 '1nlb-HL.pdb',
 '1oaq-HL-relaxed.pdb',
 '3liz-HL-true.pdb',
 '2fb4-HL-true.pdb',
 '1jpt-HL-true.pdb',
 '1dlf-HL.pdb',
 '1dlf-HL-relaxed.pdb',
 '3t65-BA.pdb',
 '2e27-HL-relaxed.pdb',
 '2r8s-HL.pdb',
 '2vxv-HL-relaxed.pdb',
 '1nlb-HL-relaxed.pdb',
 '2adf-HL-true.pdb',
 '4nzu-HL-relaxed.pdb',
 '3v0w-HL-relaxed.pdb',
 '2ypv-HL-relaxed.pdb',
 '4hpy-HL-relaxed.pdb',
 '

In [5]:
def cdr_rmsds_on_pdb_file(prediction_path, truth_path, chains):

    pred = CDR_Predictor(prediction_path, chains = chains)
    pred_coords = prepare_model_output([pred.CDR_BB_coords])[0]
    pred_coords = rearrange(pred_coords, 'i x -> () () i x')
    pred_nodefeatures = pred.prepare_model_input()[0]
    
    truth = CDR_Predictor(truth_path, chains)
    truth_coords = prepare_model_output([truth.CDR_BB_coords])[0]
    truth_coords = rearrange(truth_coords, 'i x -> () i x')

    cdr_rmsds = rmsd_per_cdr(pred_coords, pred_nodefeatures, truth_coords)

    return cdr_rmsds

def rmsds_pred_and_relaxed(dir):

    files = os.listdir(dir)
    files = [file[:7] for file in files if file[0] != '.']
    files = list(set(files))
    
    ids = list()
    rmsds_pred = np.zeros((len(files), 6))
    rmsds_relaxed = np.zeros((len(files), 6))

    for i in range(len(files)):
        chains = (files[i][-2], files[i][-1])
        
        truth_path = dir+files[i]+'-true.pdb'
        pred_path = dir+files[i]+'.pdb'
        relaxed_path = dir+files[i]+'-relaxed.pdb'

        ids.append(files[i])
        rmsds_pred[i,:] = np.array(cdr_rmsds_on_pdb_file(pred_path, truth_path, chains).tolist())
        rmsds_relaxed[i,:] = np.array(cdr_rmsds_on_pdb_file(relaxed_path, truth_path, chains).tolist())

    return ids, rmsds_pred, rmsds_relaxed

In [7]:
dir = '/Users/fabian/Desktop/2804-RAdam-5-2optim-test/'

ids, rmsd_pred, rmsd_relaxed = rmsds_pred_and_relaxed(dir)


In [8]:
rmsd_pred.mean(0)

array([1.90520121, 1.96798921, 2.92938134, 1.86615316, 1.77889952,
       1.97423427])

In [ ]:
from ABDB import database as db
from ABDB.AbPDB import AntibodyParser
import numpy as np
import Bio.PDB
parser = AntibodyParser(PERMISSIVE=True, QUIET=True)
parser.set_numbering_scheme("imgt")
db.set_numbering_scheme("imgt")
backbone = ["CA","C","N", "CB"]

def CDR_rmsds(pdb, file, fab_n = 0, chains = ["H", "L"], decoy_chains = ["H", "L"]):
    fab = db.fetch(pdb).fabs[fab_n]
    rmsds = {}
    for h_or_l in chains:
        #Truth load
        chain = db.db_summary[pdb]["fabs"][fab_n][h_or_l+"chain"]
        truth = fab.get_structure()[chain]
        #Decoy load
        decoy = parser.get_antibody_structure(pdb+"_model", file)[0][{"H":decoy_chains[0], "L":decoy_chains[1]}[h_or_l]]
        #Numbering
        numb = [(" ", *x) for x in fab.get_numbering()[h_or_l]][2:-2]
        #Get residues to align
        truth_res = [truth[x] for x in numb if (x in decoy) and (x in truth)]
        decoy_res = [decoy[x] for x in numb if (x in decoy) and (x in truth)]
        #Get atoms to align
        fixed = []
        moved = []
        for i in range(len(decoy_res)):
            fixed += [truth_res[i][atom] for atom in backbone if (atom in decoy_res[i]) and (atom in truth_res[i])]
            moved += [decoy_res[i][atom] for atom in backbone if (atom in decoy_res[i]) and (atom in truth_res[i])]
        #Calculate superimposer and move decoy
        imposer = Bio.PDB.Superimposer()
        imposer.set_atoms(fixed, moved)
        imposer.apply(decoy.get_atoms())
        rmsds[h_or_l] = imposer.rms
        # Find CDR definitions
        loop_definitions = {x[3:]:[(" ", *y[0]) for y in fab.get_CDR_sequences(definition="imgt")[x]]  for x in fab.get_CDR_sequences(definition="imgt")}
        # Calculate RMSD for each CDR
        for CDR in loop_definitions:
            if CDR[0] == h_or_l:
                true_loop = []
                decoy_loop = []
                for res in loop_definitions[CDR]:
                    if (res in truth) and (res in decoy):
                        true_loop += [truth[res][x].get_coord() for x in backbone if (x in decoy[res]) and (x in truth[res])]
                        decoy_loop+= [decoy[res][x].get_coord() for x in backbone if (x in decoy[res]) and (x in truth[res])]
                #Calculate RMSD
                rmsds[CDR] = np.sqrt(np.mean(3*(np.array(true_loop) - np.array(decoy_loop))**2))
                
    # Calculate RMSD for framework
    ignore = sum([loop_definitions[x] for x in loop_definitions if x[0] == h_or_l], [])
    frame_def = [x for x in numb if x not in ignore]
    
    true_frame, decoy_frame = [], []
    for res in frame_def:
        if (res in truth) and (res in decoy):
            true_frame += [truth[res][x].get_coord() for x in backbone if (x in decoy[res]) and (x in truth[res])]
            decoy_frame+= [decoy[res][x].get_coord() for x in backbone if (x in decoy[res]) and (x in truth[res])]
    rmsds[h_or_l] = np.sqrt(np.mean(3*(np.array(true_frame) - np.array(decoy_frame))**2))
                
    return rmsds